In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import torch
import pathlib

from torch import optim
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, Dropout, Input,Activation
from tensorflow.keras.models import Model
from keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers

import os
import glob

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !unzip -qq "/content/drive/MyDrive/cars.zip" -d "/content/drive/MyDrive/cars"
# !unzip -qq "/content/drive/MyDrive/boongboong.zip" -d "/content/drive/MyDrive/boongboong"

In [ ]:
data_dir = '/content/drive/MyDrive/boongboong' # (본인 자동차 이미지 폴더 경로)

train_ds = tf.keras.preprocessing.image_dataset_from_directory( data_dir, validation_split=0.2, subset='training', labels='inferred',
                                                               label_mode='categorical', image_size=[224, 224], seed=123, interpolation='nearest', batch_size=64, shuffle=True )

val_ds = tf.keras.preprocessing.image_dataset_from_directory( data_dir, validation_split=0.2, subset='validation', labels='inferred',
                                                             label_mode='categorical', image_size=[224, 224], seed=123, interpolation='nearest', batch_size=64, shuffle=False )

def convert_to_float(image, label):
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE 
train_ds = (train_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)) 
val_ds = (val_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))

from tensorflow.keras.applications import DenseNet121
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

num = 1
for layer in base_model.layers:
    if num >= 144:
      layer.trainable = True
    else:
      layer.trainable = False
    num += 1

model = Sequential()

model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224,224,3)))

model.add(base_model)

model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(33 , activation = 'softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

history = model.fit(train_ds,batch_size=64,epochs=40,validation_data=val_ds)


Found 9900 files belonging to 33 classes.
Using 7920 files for training.
Found 9900 files belonging to 33 classes.
Using 1980 files for validation.
Epoch 1/40
124/124 [==============================] - 82s 479ms/step - loss: 3.4883 - accuracy: 0.0362 - val_loss: 3.4853 - val_accuracy: 0.0869
Epoch 2/40
124/124 [==============================] - 46s 369ms/step - loss: 3.3358 - accuracy: 0.0718 - val_loss: 3.8364 - val_accuracy: 0.0202
Epoch 3/40
124/124 [==============================] - 46s 369ms/step - loss: 3.0398 - accuracy: 0.1223 - val_loss: 3.1252 - val_accuracy: 0.0364
Epoch 4/40
124/124 [==============================] - 46s 369ms/step - loss: 2.6309 - accuracy: 0.2128 - val_loss: 3.5153 - val_accuracy: 0.0894
Epoch 5/40
124/124 [==============================] - 46s 369ms/step - loss: 2.2939 - accuracy: 0.3119 - val_loss: 3.7326 - val_accuracy: 0.0146
Epoch 6/40
124/124 [==============================] - 46s 369ms/step - loss: 1.9911 - accuracy: 0.4091 - val_loss: 2.8820 - val

In [ ]:
model.evaluate(train_ds), model.evaluate(val_ds)

In [ ]:
data_dir2 = '/content/drive/MyDrive/cars/cars' # (본인 자동차 이미지 폴더 경로)

train_ds2 = tf.keras.preprocessing.image_dataset_from_directory( data_dir2, validation_split=0.2, subset='training', labels='inferred',
                                                               label_mode='categorical', image_size=[224, 224], seed=123, interpolation='nearest', batch_size=64, shuffle=True )

val_ds2 = tf.keras.preprocessing.image_dataset_from_directory( data_dir2, validation_split=0.2, subset='validation', labels='inferred',
                                                             label_mode='categorical', image_size=[224, 224], seed=123, interpolation='nearest', batch_size=64, shuffle=False )

def convert_to_float(image, label):
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE 
train_ds2 = (train_ds2.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)) 
val_ds2 = (val_ds2.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))

In [ ]:
model.summary()

In [ ]:
last_layer = model.get_layer('dense_1')
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dense(512,activation='relu')(x)
x = layers.Dense(256,activation='relu')(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dense(20,activation='softmax')(x)

new_model = Model(model.input, x)

In [ ]:
for layer in new_model.layers:
  layer.trainable = True

In [ ]:
new_model.compile(loss=tf.keras.losses.categorical_crossentropy, 
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
new_history = new_model.fit(train_ds2,batch_size=64,epochs=40,validation_data=val_ds2)

In [ ]:
new_model.evaluate(train_ds2), new_model.evaluate(val_ds2)

In [ ]:
# !unzip -qq "/content/drive/MyDrive/test_data.zip" -d "/content/drive/MyDrive/test_data"

In [ ]:
# Visualization
plt.style.use('fivethirtyeight')
fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,10))
axes[0].plot(new_history.history["loss"], 'r', label='Training loss')
axes[0].plot(new_history.history["val_loss"],'g',label='Validation loss' )
axes[0].set_title('Kaggle Car Classification Training and Validation Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()
plt.ylim(-0.05,1.0)
axes[1].plot (new_history.history["accuracy"],'r',label= 'Training Accuracy')
axes[1].plot (new_history.history["val_accuracy"],'g',label= 'Validation Accuracy')
axes[1].set_title('Kaggle Car Classification Training and Validation Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
 
plt.savefig("/content/drive/MyDrive/Kaggle Car Classification.jpg")

In [ ]:
# Confusion Matrix & classification report 

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.python.ops.numpy_ops import np_config


# in order to proceed following steps train_ds, val_ds dataset and  class_names should be set up 

#1. load model 
# model = tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/Base_CNN_model.keras")


#2. test data load  
#!unzip -qq "/content/drive/MyDrive/Colab Notebooks/test.zip" -d "/content/drive/MyDrive/Colab Notebooks/test"
data_dir2 = '/content/drive/MyDrive/test_data'

#3. test dataset

test_ds = tf.keras.preprocessing.image_dataset_from_directory( data_dir2, batch_size=64,
                                                               labels='inferred',
                                                               label_mode='categorical',
                                                               image_size=[224, 224],
                                                               interpolation='nearest')

#4. prediction based on trained model  
predict = model.predict(test_ds)

#5. split test dataset
(x,y),(xx,yy) = test_ds


#6. reshape y data
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior(y)
np_config.enable_numpy_behavior(yy)
y1 = y.tolist()
y2 = yy.tolist()
y3 = y1+y2
y4 = np.array(y3).reshape(99,-1)

# check shpae of y4 and predict 
y4.shape, predict.shape

#7. 
predict=np.argmax(predict, axis=1)
y4=np.argmax(y4, axis=1)
cm = confusion_matrix(y4, predict)
print(cm)

#8. precision, recall, f1 score 
from sklearn.metrics import precision_score, recall_score, f1_score
p = precision_score(y4, predict, pos_label='positive', average='micro')
print(p)
r = recall_score(y4, predict, pos_label='positive', average='micro')
print(r)
f1 = f1_score(y4, predict, pos_label='positive', average='micro')
print(f1)

clas_names = test_ds.class_names
#9. classification report 
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y4, predict, target_names=class_names))

#10. confusion matrix visualization
class_count = len(class_names)

import seaborn as sns
plt.figure(figsize=(15, 10))
sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
plt.xticks(np.arange(class_count)+.5, class_names, rotation= 90)
plt.yticks(np.arange(class_count)+.5, class_names, rotation=0)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Base CNN Model Confusion Matrix")
plt.savefig('/content/drive/MyDrive/Colab Notebooks/baseCNN_model_confusion_matrix.jpg')
plt.show()

In [ ]:
from sklearn import metrics
confusion_matrix = metrics.confusion_matrix( )
print(confusion_matrix)